In [24]:
import psycopg2
import psycopg2.extras as extras

import requests
import time
import pandas as pd
import numpy as np

In [3]:
# access source RDS source database (datalake)
try:
    conn = psycopg2.connect("host=aoe21.cob0rzy4a66o.us-east-1.rds.amazonaws.com dbname=aoe21 user=tico password=1234567890")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
# establish cursor for source
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
conn.set_session(autocommit = True)

In [4]:
# access target RDS target database (datawarehouse)
try:
    conn_target = psycopg2.connect("host=aoe2warehouse.cm7nj2633ign.us-east-1.rds.amazonaws.com dbname=aoe2warehouse user=pasci password=123456789")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
# establish cursor for target
try:
    cur_target = conn_target.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
conn_target.set_session(autocommit = True)

In [8]:
postgreSQL_select_Query = "SELECT * FROM match_history"
cur.execute(postgreSQL_select_Query)

#Select all data from table player_count
match_history = cur.fetchall()
match_history = pd.DataFrame(match_history)

In [9]:
match_history.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,150510221,72,ukwest,1648114790,1.648117e+09,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,16.0,-16.0,15.0,13.0,True,False
1,150510192,9,ukwest,1648114796,1.648117e+09,7.656120e+16,7.656120e+16,US,DK,732.0,NaN,-13.0,NaN,2.0,27.0,False,True
2,150510274,9,ukwest,1648114808,1.648117e+09,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,17.0,NaN,18.0,25.0,True,False
3,150510276,77,ukwest,1648114816,1.648116e+09,NaN,7.656120e+16,None,FI,1107.0,1129.0,-14.0,14.0,32.0,35.0,False,True
4,150510270,29,southeastasia,1648114812,1.648116e+09,7.656120e+16,7.656120e+16,IN,CN,530.0,670.0,-9.0,13.0,3.0,2.0,False,True


In [16]:
match_history.columns =['match_id', 'map', 'server', 'started', 'finished', 'steam_id_a', 'steam_id_b', 'country_a', 'country_b', 'rating_a', 'rating_b', 'rating_change_a', 'rating_change_b', 'civilization_a', 'civilization_b', 'win_a', 'win_b']

In [17]:
match_history.head()

,match_id,map,server,started,finished,steam_id_a,steam_id_b,country_a,country_b,rating_a,rating_b,rating_change_a,rating_change_b,civilization_a,civilization_b,win_a,win_b
0,150510221,72,ukwest,1648114790,1.648117e+09,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,16.0,-16.0,15.0,13.0,True,False
1,150510192,9,ukwest,1648114796,1.648117e+09,7.656120e+16,7.656120e+16,US,DK,732.0,NaN,-13.0,NaN,2.0,27.0,False,True
2,150510274,9,ukwest,1648114808,1.648117e+09,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,17.0,NaN,18.0,25.0,True,False
3,150510276,77,ukwest,1648114816,1.648116e+09,NaN,7.656120e+16,None,FI,1107.0,1129.0,-14.0,14.0,32.0,35.0,False,True
4,150510270,29,southeastasia,1648114812,1.648116e+09,7.656120e+16,7.656120e+16,IN,CN,530.0,670.0,-9.0,13.0,3.0,2.0,False,True


In [46]:
# create dublicate
match_history_adjusted = match_history

In [47]:
# rename map to map_id
match_history.rename(columns = {"map": "map_id"}, inplace = True)

In [48]:
# delete matches with no winner or steam id
# those matches are not suitable for the visualization
match_history_adjusted = match_history_adjusted.dropna(axis = 0, subset = ["win_a"])
match_history_adjusted = match_history_adjusted.dropna(axis = 0, subset = ["win_b"])
match_history_adjusted = match_history_adjusted.dropna(axis = 0, subset = ["steam_id_a"])
match_history_adjusted = match_history_adjusted.dropna(axis = 0, subset = ["steam_id_b"])

In [49]:
# drop rating_change_a and rating_change_b
# information not suitable for the visualization
match_history_adjusted = match_history_adjusted.drop(["rating_change_a", "rating_change_b"], axis = 1)

In [50]:
# categorize matchups into winner and loser information instead of a and b
match_history_adjusted["steam_id_win"] = match_history_adjusted.apply(lambda x: x["steam_id_a"] if x["win_a"] == True else x["steam_id_b"], axis = 1)
match_history_adjusted["steam_id_lose"] = match_history_adjusted.apply(lambda x: x["steam_id_b"] if x["win_a"] == True else x["steam_id_a"], axis = 1)
match_history_adjusted["country_win"] = match_history_adjusted.apply(lambda x: x["country_a"] if x["win_a"] == True else x["country_b"], axis = 1)
match_history_adjusted["country_lose"] = match_history_adjusted.apply(lambda x: x["country_b"] if x["win_a"] == True else x["country_a"], axis = 1)
match_history_adjusted["rating_win"] = match_history_adjusted.apply(lambda x: x["rating_a"] if x["win_a"] == True else x["rating_b"], axis = 1)
match_history_adjusted["rating_lose"] = match_history_adjusted.apply(lambda x: x["rating_b"] if x["win_a"] == True else x["rating_a"], axis = 1)
match_history_adjusted["civilization_win"] = match_history_adjusted.apply(lambda x: x["civilization_a"] if x["win_a"] == True else x["civilization_b"], axis = 1)
match_history_adjusted["civilization_lose"] = match_history_adjusted.apply(lambda x: x["civilization_b"] if x["win_a"] == True else x["civilization_a"], axis = 1)

In [51]:
request = requests.get("https://aoe2.net/api/strings?game=aoe2de&language=en")
request = request.json()
request

{'language': 'en',
 'age': [{'id': 0, 'string': 'Standard'},
  {'id': 2, 'string': 'Dark Age'},
  {'id': 3, 'string': 'Feudal Age'},
  {'id': 4, 'string': 'Castle Age'},
  {'id': 5, 'string': 'Imperial Age'},
  {'id': 6, 'string': 'Post-Imperial Age'}],
 'civ': [{'id': 1, 'string': 'Britons'},
  {'id': 2, 'string': 'Franks'},
  {'id': 3, 'string': 'Goths'},
  {'id': 4, 'string': 'Teutons'},
  {'id': 5, 'string': 'Japanese'},
  {'id': 6, 'string': 'Chinese'},
  {'id': 7, 'string': 'Byzantines'},
  {'id': 8, 'string': 'Persians'},
  {'id': 9, 'string': 'Saracens'},
  {'id': 10, 'string': 'Turks'},
  {'id': 11, 'string': 'Vikings'},
  {'id': 12, 'string': 'Mongols'},
  {'id': 13, 'string': 'Celts'},
  {'id': 14, 'string': 'Spanish'},
  {'id': 15, 'string': 'Aztecs'},
  {'id': 16, 'string': 'Mayans'},
  {'id': 17, 'string': 'Huns'},
  {'id': 18, 'string': 'Koreans'},
  {'id': 19, 'string': 'Italians'},
  {'id': 20, 'string': 'Hindustanis'},
  {'id': 21, 'string': 'Incas'},
  {'id': 22, 'st

In [52]:
# read in civilization keys
civilizations = request["civ"]
civilizations = pd.DataFrame.from_dict(civilizations)
civilizations

,id,string
0,1,Britons
1,2,Franks
2,3,Goths
3,4,Teutons
4,5,Japanese
5,6,Chinese
6,7,Byzantines
7,8,Persians
8,9,Saracens
9,10,Turks


In [53]:
# replace civilization key with civilization name for civilization_win
match_history_adjusted = pd.merge(match_history_adjusted, civilizations, how = "left", left_on = "civilization_win", right_on = "id")
match_history_adjusted = match_history_adjusted.drop(["civilization_win", "id"], axis = 1)
match_history_adjusted = match_history_adjusted.rename(columns = {"string": "civilization_win"})
match_history_adjusted

,match_id,map_id,server,started,finished,steam_id_a,steam_id_b,country_a,country_b,rating_a,...,win_a,win_b,steam_id_win,steam_id_lose,country_win,country_lose,rating_win,rating_lose,civilization_lose,civilization_win
0,150510221,72,ukwest,1648114790,1.648117e+09,7.656120e+16,7.656120e+16,EE,RO,883.0,...,True,False,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,13.0,Aztecs
1,150510192,9,ukwest,1648114796,1.648117e+09,7.656120e+16,7.656120e+16,US,DK,732.0,...,False,True,7.656120e+16,7.656120e+16,DK,US,NaN,732.0,2.0,Berbers
2,150510274,9,ukwest,1648114808,1.648117e+09,7.656120e+16,7.656120e+16,FR,DE,1144.0,...,True,False,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,25.0,Koreans
3,150510270,29,southeastasia,1648114812,1.648116e+09,7.656120e+16,7.656120e+16,IN,CN,530.0,...,False,True,7.656120e+16,7.656120e+16,CN,IN,670.0,530.0,3.0,Franks
4,150510263,77,westindia,1648114820,1.648116e+09,7.656120e+16,7.656120e+16,DE,TW,1379.0,...,False,True,7.656120e+16,7.656120e+16,TW,DE,1388.0,1379.0,14.0,Ethiopians
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447324,155880960,29,eastus,1650745770,1.650746e+09,7.656120e+16,7.656120e+16,DO,JP,1493.0,...,True,False,7.656120e+16,7.656120e+16,DO,JP,1493.0,1504.0,8.0,Persians
447325,155881049,19,brazilsouth,1650745782,1.650748e+09,7.656120e+16,7.656120e+16,BR,AR,800.0,...,False,True,7.656120e+16,7.656120e+16,AR,BR,856.0,800.0,24.0,Malay
447326,155881067,9,ukwest,1650745770,1.650747e+09,7.656120e+16,7.656120e+16,ES,DE,1156.0,...,False,True,7.656120e+16,7.656120e+16,DE,ES,1153.0,1156.0,29.0,Franks
447327,155881191,9,ukwest,1650745797,1.650747e+09,7.656120e+16,7.656120e+16,CZ,DE,2214.0,...,False,True,7.656120e+16,7.656120e+16,DE,CZ,2239.0,2214.0,18.0,Goths


In [54]:
# replace civilization key with civilization name for civilization_lose
match_history_adjusted = pd.merge(match_history_adjusted, civilizations, how = "left", left_on = "civilization_lose", right_on = "id")
match_history_adjusted = match_history_adjusted.drop(["civilization_lose", "id"], axis = 1)
match_history_adjusted = match_history_adjusted.rename(columns = {"string": "civilization_lose"})
match_history_adjusted

,match_id,map_id,server,started,finished,steam_id_a,steam_id_b,country_a,country_b,rating_a,...,win_a,win_b,steam_id_win,steam_id_lose,country_win,country_lose,rating_win,rating_lose,civilization_win,civilization_lose
0,150510221,72,ukwest,1648114790,1.648117e+09,7.656120e+16,7.656120e+16,EE,RO,883.0,...,True,False,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,Aztecs,Celts
1,150510192,9,ukwest,1648114796,1.648117e+09,7.656120e+16,7.656120e+16,US,DK,732.0,...,False,True,7.656120e+16,7.656120e+16,DK,US,NaN,732.0,Berbers,Franks
2,150510274,9,ukwest,1648114808,1.648117e+09,7.656120e+16,7.656120e+16,FR,DE,1144.0,...,True,False,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,Koreans,Ethiopians
3,150510270,29,southeastasia,1648114812,1.648116e+09,7.656120e+16,7.656120e+16,IN,CN,530.0,...,False,True,7.656120e+16,7.656120e+16,CN,IN,670.0,530.0,Franks,Goths
4,150510263,77,westindia,1648114820,1.648116e+09,7.656120e+16,7.656120e+16,DE,TW,1379.0,...,False,True,7.656120e+16,7.656120e+16,TW,DE,1388.0,1379.0,Ethiopians,Spanish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447324,155880960,29,eastus,1650745770,1.650746e+09,7.656120e+16,7.656120e+16,DO,JP,1493.0,...,True,False,7.656120e+16,7.656120e+16,DO,JP,1493.0,1504.0,Persians,Persians
447325,155881049,19,brazilsouth,1650745782,1.650748e+09,7.656120e+16,7.656120e+16,BR,AR,800.0,...,False,True,7.656120e+16,7.656120e+16,AR,BR,856.0,800.0,Malay,Portuguese
447326,155881067,9,ukwest,1650745770,1.650747e+09,7.656120e+16,7.656120e+16,ES,DE,1156.0,...,False,True,7.656120e+16,7.656120e+16,DE,ES,1153.0,1156.0,Franks,Malay
447327,155881191,9,ukwest,1650745797,1.650747e+09,7.656120e+16,7.656120e+16,CZ,DE,2214.0,...,False,True,7.656120e+16,7.656120e+16,DE,CZ,2239.0,2214.0,Goths,Koreans


In [55]:
# read in map keys
maps = request["map_type"]
maps = pd.DataFrame.from_dict(maps)
maps

,id,string
0,9,Arabia
1,10,Archipelago
2,11,Baltic
3,12,Black Forest
4,13,Coastal
...,...,...
138,170,Haboob
139,171,Kawasan
140,172,Land Madness
141,173,Sacred Springs


In [56]:
# replace map key with map name
match_history_adjusted = pd.merge(match_history_adjusted, maps, how = "left", left_on = "map_id", right_on = "id")
match_history_adjusted = match_history_adjusted.drop(["map_id", "id"], axis = 1)
match_history_adjusted = match_history_adjusted.rename(columns = {"string": "map"})
match_history_adjusted

,match_id,server,started,finished,steam_id_a,steam_id_b,country_a,country_b,rating_a,rating_b,...,win_b,steam_id_win,steam_id_lose,country_win,country_lose,rating_win,rating_lose,civilization_win,civilization_lose,map
0,150510221,ukwest,1648114790,1.648117e+09,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,...,False,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,Aztecs,Celts,Hideout
1,150510192,ukwest,1648114796,1.648117e+09,7.656120e+16,7.656120e+16,US,DK,732.0,NaN,...,True,7.656120e+16,7.656120e+16,DK,US,NaN,732.0,Berbers,Franks,Arabia
2,150510274,ukwest,1648114808,1.648117e+09,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,...,False,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,Koreans,Ethiopians,Arabia
3,150510270,southeastasia,1648114812,1.648116e+09,7.656120e+16,7.656120e+16,IN,CN,530.0,670.0,...,True,7.656120e+16,7.656120e+16,CN,IN,670.0,530.0,Franks,Goths,Arena
4,150510263,westindia,1648114820,1.648116e+09,7.656120e+16,7.656120e+16,DE,TW,1379.0,1388.0,...,True,7.656120e+16,7.656120e+16,TW,DE,1388.0,1379.0,Ethiopians,Spanish,MegaRandom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447324,155880960,eastus,1650745770,1.650746e+09,7.656120e+16,7.656120e+16,DO,JP,1493.0,1504.0,...,False,7.656120e+16,7.656120e+16,DO,JP,1493.0,1504.0,Persians,Persians,Arena
447325,155881049,brazilsouth,1650745782,1.650748e+09,7.656120e+16,7.656120e+16,BR,AR,800.0,856.0,...,True,7.656120e+16,7.656120e+16,AR,BR,856.0,800.0,Malay,Portuguese,Islands
447326,155881067,ukwest,1650745770,1.650747e+09,7.656120e+16,7.656120e+16,ES,DE,1156.0,1153.0,...,True,7.656120e+16,7.656120e+16,DE,ES,1153.0,1156.0,Franks,Malay,Arabia
447327,155881191,ukwest,1650745797,1.650747e+09,7.656120e+16,7.656120e+16,CZ,DE,2214.0,2239.0,...,True,7.656120e+16,7.656120e+16,DE,CZ,2239.0,2214.0,Goths,Koreans,Arabia


In [57]:
# decode country keys via additional api
request = requests.get("https://restcountries.com/v3.1/all")
request = request.json()
request

[{'name': {'common': 'Germany',
   'official': 'Federal Republic of Germany',
   'nativeName': {'deu': {'official': 'Bundesrepublik Deutschland',
     'common': 'Deutschland'}}},
  'tld': ['.de'],
  'cca2': 'DE',
  'ccn3': '276',
  'cca3': 'DEU',
  'cioc': 'GER',
  'independent': True,
  'status': 'officially-assigned',
  'unMember': True,
  'currencies': {'EUR': {'name': 'Euro', 'symbol': '€'}},
  'idd': {'root': '+4', 'suffixes': ['9']},
  'capital': ['Berlin'],
  'altSpellings': ['DE',
   'Federal Republic of Germany',
   'Bundesrepublik Deutschland'],
  'region': 'Europe',
  'subregion': 'Western Europe',
  'languages': {'deu': 'German'},
  'translations': {'ara': {'official': 'جمهورية ألمانيا الاتحادية',
    'common': 'ألمانيا'},
   'ces': {'official': 'Spolková republika Německo', 'common': 'Německo'},
   'cym': {'official': 'Federal Republic of Germany', 'common': 'Germany'},
   'deu': {'official': 'Bundesrepublik Deutschland', 'common': 'Deutschland'},
   'est': {'official': 'S

In [58]:
# read in country keys
countries_common_name = [d["name"]["common"] for d in request]
countries_cca2 = [d["cca2"] for d in request]
countries = {"common_name": countries_common_name, "cca2": countries_cca2}
countries = pd.DataFrame(countries)
countries

,common_name,cca2
0,Germany,DE
1,Morocco,MA
2,Belgium,BE
3,Cook Islands,CK
4,Ecuador,EC
...,...,...
245,Bermuda,BM
246,Slovakia,SK
247,Bangladesh,BD
248,Denmark,DK


In [59]:
# replace country key with country name for country_win
match_history_adjusted = pd.merge(match_history_adjusted, countries, how = "left", left_on = "country_win", right_on = "cca2")
match_history_adjusted = match_history_adjusted.drop(["country_win", "cca2"], axis = 1)
match_history_adjusted = match_history_adjusted.rename(columns = {"common_name": "country_win"})
match_history_adjusted

,match_id,server,started,finished,steam_id_a,steam_id_b,country_a,country_b,rating_a,rating_b,...,win_b,steam_id_win,steam_id_lose,country_lose,rating_win,rating_lose,civilization_win,civilization_lose,map,country_win
0,150510221,ukwest,1648114790,1.648117e+09,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,...,False,7.656120e+16,7.656120e+16,RO,883.0,898.0,Aztecs,Celts,Hideout,Estonia
1,150510192,ukwest,1648114796,1.648117e+09,7.656120e+16,7.656120e+16,US,DK,732.0,NaN,...,True,7.656120e+16,7.656120e+16,US,NaN,732.0,Berbers,Franks,Arabia,Denmark
2,150510274,ukwest,1648114808,1.648117e+09,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,...,False,7.656120e+16,7.656120e+16,DE,1144.0,1176.0,Koreans,Ethiopians,Arabia,France
3,150510270,southeastasia,1648114812,1.648116e+09,7.656120e+16,7.656120e+16,IN,CN,530.0,670.0,...,True,7.656120e+16,7.656120e+16,IN,670.0,530.0,Franks,Goths,Arena,China
4,150510263,westindia,1648114820,1.648116e+09,7.656120e+16,7.656120e+16,DE,TW,1379.0,1388.0,...,True,7.656120e+16,7.656120e+16,DE,1388.0,1379.0,Ethiopians,Spanish,MegaRandom,Taiwan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447324,155880960,eastus,1650745770,1.650746e+09,7.656120e+16,7.656120e+16,DO,JP,1493.0,1504.0,...,False,7.656120e+16,7.656120e+16,JP,1493.0,1504.0,Persians,Persians,Arena,Dominican Republic
447325,155881049,brazilsouth,1650745782,1.650748e+09,7.656120e+16,7.656120e+16,BR,AR,800.0,856.0,...,True,7.656120e+16,7.656120e+16,BR,856.0,800.0,Malay,Portuguese,Islands,Argentina
447326,155881067,ukwest,1650745770,1.650747e+09,7.656120e+16,7.656120e+16,ES,DE,1156.0,1153.0,...,True,7.656120e+16,7.656120e+16,ES,1153.0,1156.0,Franks,Malay,Arabia,Germany
447327,155881191,ukwest,1650745797,1.650747e+09,7.656120e+16,7.656120e+16,CZ,DE,2214.0,2239.0,...,True,7.656120e+16,7.656120e+16,CZ,2239.0,2214.0,Goths,Koreans,Arabia,Germany


In [60]:
# replace country key with country name for country_lose
match_history_adjusted = pd.merge(match_history_adjusted, countries, how = "left", left_on = "country_lose", right_on = "cca2")
match_history_adjusted = match_history_adjusted.drop(["country_lose", "cca2"], axis = 1)
match_history_adjusted = match_history_adjusted.rename(columns = {"common_name": "country_lose"})
match_history_adjusted

,match_id,server,started,finished,steam_id_a,steam_id_b,country_a,country_b,rating_a,rating_b,...,win_b,steam_id_win,steam_id_lose,rating_win,rating_lose,civilization_win,civilization_lose,map,country_win,country_lose
0,150510221,ukwest,1648114790,1.648117e+09,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,...,False,7.656120e+16,7.656120e+16,883.0,898.0,Aztecs,Celts,Hideout,Estonia,Romania
1,150510192,ukwest,1648114796,1.648117e+09,7.656120e+16,7.656120e+16,US,DK,732.0,NaN,...,True,7.656120e+16,7.656120e+16,NaN,732.0,Berbers,Franks,Arabia,Denmark,United States
2,150510274,ukwest,1648114808,1.648117e+09,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,...,False,7.656120e+16,7.656120e+16,1144.0,1176.0,Koreans,Ethiopians,Arabia,France,Germany
3,150510270,southeastasia,1648114812,1.648116e+09,7.656120e+16,7.656120e+16,IN,CN,530.0,670.0,...,True,7.656120e+16,7.656120e+16,670.0,530.0,Franks,Goths,Arena,China,India
4,150510263,westindia,1648114820,1.648116e+09,7.656120e+16,7.656120e+16,DE,TW,1379.0,1388.0,...,True,7.656120e+16,7.656120e+16,1388.0,1379.0,Ethiopians,Spanish,MegaRandom,Taiwan,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447324,155880960,eastus,1650745770,1.650746e+09,7.656120e+16,7.656120e+16,DO,JP,1493.0,1504.0,...,False,7.656120e+16,7.656120e+16,1493.0,1504.0,Persians,Persians,Arena,Dominican Republic,Japan
447325,155881049,brazilsouth,1650745782,1.650748e+09,7.656120e+16,7.656120e+16,BR,AR,800.0,856.0,...,True,7.656120e+16,7.656120e+16,856.0,800.0,Malay,Portuguese,Islands,Argentina,Brazil
447326,155881067,ukwest,1650745770,1.650747e+09,7.656120e+16,7.656120e+16,ES,DE,1156.0,1153.0,...,True,7.656120e+16,7.656120e+16,1153.0,1156.0,Franks,Malay,Arabia,Germany,Spain
447327,155881191,ukwest,1650745797,1.650747e+09,7.656120e+16,7.656120e+16,CZ,DE,2214.0,2239.0,...,True,7.656120e+16,7.656120e+16,2239.0,2214.0,Goths,Koreans,Arabia,Germany,Czechia


In [61]:
# calculate match duration in minutes
match_history_adjusted["duration_min"] = match_history_adjusted["finished"] - match_history_adjusted["started"]
match_history_adjusted["duration_min"] = match_history_adjusted["duration_min"] / 60
match_history_adjusted["duration_min"] = round(match_history_adjusted["duration_min"])
match_history_adjusted

,match_id,server,started,finished,steam_id_a,steam_id_b,country_a,country_b,rating_a,rating_b,...,steam_id_win,steam_id_lose,rating_win,rating_lose,civilization_win,civilization_lose,map,country_win,country_lose,duration_min
0,150510221,ukwest,1648114790,1.648117e+09,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,...,7.656120e+16,7.656120e+16,883.0,898.0,Aztecs,Celts,Hideout,Estonia,Romania,30.0
1,150510192,ukwest,1648114796,1.648117e+09,7.656120e+16,7.656120e+16,US,DK,732.0,NaN,...,7.656120e+16,7.656120e+16,NaN,732.0,Berbers,Franks,Arabia,Denmark,United States,32.0
2,150510274,ukwest,1648114808,1.648117e+09,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,...,7.656120e+16,7.656120e+16,1144.0,1176.0,Koreans,Ethiopians,Arabia,France,Germany,32.0
3,150510270,southeastasia,1648114812,1.648116e+09,7.656120e+16,7.656120e+16,IN,CN,530.0,670.0,...,7.656120e+16,7.656120e+16,670.0,530.0,Franks,Goths,Arena,China,India,21.0
4,150510263,westindia,1648114820,1.648116e+09,7.656120e+16,7.656120e+16,DE,TW,1379.0,1388.0,...,7.656120e+16,7.656120e+16,1388.0,1379.0,Ethiopians,Spanish,MegaRandom,Taiwan,Germany,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447324,155880960,eastus,1650745770,1.650746e+09,7.656120e+16,7.656120e+16,DO,JP,1493.0,1504.0,...,7.656120e+16,7.656120e+16,1493.0,1504.0,Persians,Persians,Arena,Dominican Republic,Japan,4.0
447325,155881049,brazilsouth,1650745782,1.650748e+09,7.656120e+16,7.656120e+16,BR,AR,800.0,856.0,...,7.656120e+16,7.656120e+16,856.0,800.0,Malay,Portuguese,Islands,Argentina,Brazil,35.0
447326,155881067,ukwest,1650745770,1.650747e+09,7.656120e+16,7.656120e+16,ES,DE,1156.0,1153.0,...,7.656120e+16,7.656120e+16,1153.0,1156.0,Franks,Malay,Arabia,Germany,Spain,14.0
447327,155881191,ukwest,1650745797,1.650747e+09,7.656120e+16,7.656120e+16,CZ,DE,2214.0,2239.0,...,7.656120e+16,7.656120e+16,2239.0,2214.0,Goths,Koreans,Arabia,Germany,Czechia,19.0


In [62]:
# unix_time_stamp to datetime for started
match_history_adjusted["started_datetime"] = pd.to_datetime(match_history_adjusted["started"], unit = "s")
match_history_adjusted = match_history_adjusted.drop(["started"], axis = 1)
match_history_adjusted

,match_id,server,finished,steam_id_a,steam_id_b,country_a,country_b,rating_a,rating_b,civilization_a,...,steam_id_lose,rating_win,rating_lose,civilization_win,civilization_lose,map,country_win,country_lose,duration_min,started_datetime
0,150510221,ukwest,1.648117e+09,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,15.0,...,7.656120e+16,883.0,898.0,Aztecs,Celts,Hideout,Estonia,Romania,30.0,2022-03-24 09:39:50
1,150510192,ukwest,1.648117e+09,7.656120e+16,7.656120e+16,US,DK,732.0,NaN,2.0,...,7.656120e+16,NaN,732.0,Berbers,Franks,Arabia,Denmark,United States,32.0,2022-03-24 09:39:56
2,150510274,ukwest,1.648117e+09,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,18.0,...,7.656120e+16,1144.0,1176.0,Koreans,Ethiopians,Arabia,France,Germany,32.0,2022-03-24 09:40:08
3,150510270,southeastasia,1.648116e+09,7.656120e+16,7.656120e+16,IN,CN,530.0,670.0,3.0,...,7.656120e+16,670.0,530.0,Franks,Goths,Arena,China,India,21.0,2022-03-24 09:40:12
4,150510263,westindia,1.648116e+09,7.656120e+16,7.656120e+16,DE,TW,1379.0,1388.0,14.0,...,7.656120e+16,1388.0,1379.0,Ethiopians,Spanish,MegaRandom,Taiwan,Germany,21.0,2022-03-24 09:40:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447324,155880960,eastus,1.650746e+09,7.656120e+16,7.656120e+16,DO,JP,1493.0,1504.0,8.0,...,7.656120e+16,1493.0,1504.0,Persians,Persians,Arena,Dominican Republic,Japan,4.0,2022-04-23 20:29:30
447325,155881049,brazilsouth,1.650748e+09,7.656120e+16,7.656120e+16,BR,AR,800.0,856.0,24.0,...,7.656120e+16,856.0,800.0,Malay,Portuguese,Islands,Argentina,Brazil,35.0,2022-04-23 20:29:42
447326,155881067,ukwest,1.650747e+09,7.656120e+16,7.656120e+16,ES,DE,1156.0,1153.0,29.0,...,7.656120e+16,1153.0,1156.0,Franks,Malay,Arabia,Germany,Spain,14.0,2022-04-23 20:29:30
447327,155881191,ukwest,1.650747e+09,7.656120e+16,7.656120e+16,CZ,DE,2214.0,2239.0,18.0,...,7.656120e+16,2239.0,2214.0,Goths,Koreans,Arabia,Germany,Czechia,19.0,2022-04-23 20:29:57


In [63]:
# unix_time_stamp to datetime for finished
match_history_adjusted["finished_datetime"] = pd.to_datetime(match_history_adjusted["finished"], unit = "s")
match_history_adjusted = match_history_adjusted.drop(["finished"], axis = 1)
match_history_adjusted

,match_id,server,steam_id_a,steam_id_b,country_a,country_b,rating_a,rating_b,civilization_a,civilization_b,...,rating_win,rating_lose,civilization_win,civilization_lose,map,country_win,country_lose,duration_min,started_datetime,finished_datetime
0,150510221,ukwest,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,15.0,13.0,...,883.0,898.0,Aztecs,Celts,Hideout,Estonia,Romania,30.0,2022-03-24 09:39:50,2022-03-24 10:09:44
1,150510192,ukwest,7.656120e+16,7.656120e+16,US,DK,732.0,NaN,2.0,27.0,...,NaN,732.0,Berbers,Franks,Arabia,Denmark,United States,32.0,2022-03-24 09:39:56,2022-03-24 10:12:16
2,150510274,ukwest,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,18.0,25.0,...,1144.0,1176.0,Koreans,Ethiopians,Arabia,France,Germany,32.0,2022-03-24 09:40:08,2022-03-24 10:11:50
3,150510270,southeastasia,7.656120e+16,7.656120e+16,IN,CN,530.0,670.0,3.0,2.0,...,670.0,530.0,Franks,Goths,Arena,China,India,21.0,2022-03-24 09:40:12,2022-03-24 10:01:31
4,150510263,westindia,7.656120e+16,7.656120e+16,DE,TW,1379.0,1388.0,14.0,25.0,...,1388.0,1379.0,Ethiopians,Spanish,MegaRandom,Taiwan,Germany,21.0,2022-03-24 09:40:20,2022-03-24 10:01:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447324,155880960,eastus,7.656120e+16,7.656120e+16,DO,JP,1493.0,1504.0,8.0,8.0,...,1493.0,1504.0,Persians,Persians,Arena,Dominican Republic,Japan,4.0,2022-04-23 20:29:30,2022-04-23 20:33:42
447325,155881049,brazilsouth,7.656120e+16,7.656120e+16,BR,AR,800.0,856.0,24.0,29.0,...,856.0,800.0,Malay,Portuguese,Islands,Argentina,Brazil,35.0,2022-04-23 20:29:42,2022-04-23 21:05:06
447326,155881067,ukwest,7.656120e+16,7.656120e+16,ES,DE,1156.0,1153.0,29.0,2.0,...,1153.0,1156.0,Franks,Malay,Arabia,Germany,Spain,14.0,2022-04-23 20:29:30,2022-04-23 20:43:48
447327,155881191,ukwest,7.656120e+16,7.656120e+16,CZ,DE,2214.0,2239.0,18.0,3.0,...,2239.0,2214.0,Goths,Koreans,Arabia,Germany,Czechia,19.0,2022-04-23 20:29:57,2022-04-23 20:49:25


In [71]:
# convert datatypes
match_history_adjusted["steam_id_win"] = match_history_adjusted["steam_id_win"].astype("Int64")
match_history_adjusted["steam_id_lose"] = match_history_adjusted["steam_id_lose"].astype("Int64")
#match_history_adjusted["rating_win"] = match_history_adjusted["rating_win"].fillna(0).astype("Int64")
#match_history_adjusted["rating_lose"] = match_history_adjusted["rating_lose"].fillna(0).astype("Int64")
match_history_adjusted["duration_min"] = match_history_adjusted["duration_min"].astype("Int64")
match_history_adjusted.head()

,match_id,server,steam_id_a,steam_id_b,country_a,country_b,rating_a,rating_b,civilization_a,civilization_b,...,rating_win,rating_lose,civilization_win,civilization_lose,map,country_win,country_lose,duration_min,started_datetime,finished_datetime
0,150510221,ukwest,7.656120e+16,7.656120e+16,EE,RO,883.0,898.0,15.0,13.0,...,883.0,898.0,Aztecs,Celts,Hideout,Estonia,Romania,30,2022-03-24 09:39:50,2022-03-24 10:09:44
1,150510192,ukwest,7.656120e+16,7.656120e+16,US,DK,732.0,NaN,2.0,27.0,...,NaN,732.0,Berbers,Franks,Arabia,Denmark,United States,32,2022-03-24 09:39:56,2022-03-24 10:12:16
2,150510274,ukwest,7.656120e+16,7.656120e+16,FR,DE,1144.0,1176.0,18.0,25.0,...,1144.0,1176.0,Koreans,Ethiopians,Arabia,France,Germany,32,2022-03-24 09:40:08,2022-03-24 10:11:50
3,150510270,southeastasia,7.656120e+16,7.656120e+16,IN,CN,530.0,670.0,3.0,2.0,...,670.0,530.0,Franks,Goths,Arena,China,India,21,2022-03-24 09:40:12,2022-03-24 10:01:31
4,150510263,westindia,7.656120e+16,7.656120e+16,DE,TW,1379.0,1388.0,14.0,25.0,...,1388.0,1379.0,Ethiopians,Spanish,MegaRandom,Taiwan,Germany,21,2022-03-24 09:40:20,2022-03-24 10:01:18


In [101]:
match_history_adjusted.min()

C:\Users\pasci\AppData\Local\Temp\ipykernel_26584\127898238.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  match_history_adjusted.min()


match_id                       150510192
steam_id_a           76561197960279824.0
steam_id_b           76561197960279824.0
rating_a                             0.0
rating_b                            58.0
civilization_a                       1.0
civilization_b                       1.0
win_a                              False
win_b                              False
steam_id_win           76561197960279824
steam_id_lose          76561197960279824
rating_win                          56.0
rating_lose                          0.0
civilization_lose                 Aztecs
map                     African Clearing
duration_min                           0
started_datetime     2022-03-24 09:39:50
finished_datetime    2022-03-24 09:43:17
dtype: object

In [65]:
list(match_history_adjusted.columns.values)

['match_id',
 'server',
 'steam_id_a',
 'steam_id_b',
 'country_a',
 'country_b',
 'rating_a',
 'rating_b',
 'civilization_a',
 'civilization_b',
 'win_a',
 'win_b',
 'steam_id_win',
 'steam_id_lose',
 'rating_win',
 'rating_lose',
 'civilization_win',
 'civilization_lose',
 'map',
 'country_win',
 'country_lose',
 'duration_min',
 'started_datetime',
 'finished_datetime']

In [102]:
cur_target.execute("DROP TABLE match_history;")

In [103]:
#Create Table if it not exists
cur_target.execute("CREATE TABLE IF NOT EXISTS match_history (match_id int, server varchar(100), steam_id_a bigint, steam_id_b bigint, country_a varchar(100), country_b varchar(100), rating_a varchar(100), rating_b varchar(100), civilization_a int, civilization_b int, win_a bool, win_b bool, steam_id_win bigint ,steam_id_lose bigint, rating_win varchar(100), rating_lose varchar(100), civilization_win varchar(100), civilization_lose varchar(100), map varchar(100), country_win varchar(100), country_lose varchar(100), duration_min int, started_datetime timestamp, finished_datetime timestamp);")

In [104]:
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()

In [105]:
execute_values(conn_target, match_history_adjusted, 'match_history')

the dataframe is inserted
